# Análisis Estadístico


Análisis del rendimiento general de cada equipo a lo largo de las temporadas desde 2016 hasta 2023. Incluiré el total de victorias derrotas, empates, goles a favor, goles en contra, y el porcentaje de victorias, empates y derrotas.

In [5]:
import pandas as pd

archivos = [
    'datos_limpios/champions_2016_2017.csv',
    'datos_limpios/champions_2017_2018.csv',
    'datos_limpios/champions_2018_2019.csv',
    'datos_limpios/champions_2019_2020.csv',
    'datos_limpios/champions_2020_2021.csv',
    'datos_limpios/champions_2021_2022.csv',
    'datos_limpios/champions_2022_2023.csv'
]

#Leer y almacenar el contenido de los archivos
dfs = [pd.read_csv(archivos) for archivos in archivos]

#Verificar primeras filas de cada df para ver estructura
dfs_vista = [df.head() for df in dfs]
dfs_vista[0]


,SEASON,HOME_TEAM,AWAY_TEAM,HT_SCORE,AW_SCORE
0,2016,FC Basel,PFC Ludogorets Razgrad,1,1
1,2016,Paris Saint-Germain,Arsenal FC,1,1
2,2016,Arsenal FC,FC Basel,2,0
3,2016,PFC Ludogorets Razgrad,Paris Saint-Germain,1,3
4,2016,Arsenal FC,PFC Ludogorets Razgrad,6,0


In [7]:
#Unir dataframes en uno
df_todos = pd.concat(dfs, ignore_index=True)

#Funcion para determinar el resultado de cada partido
def resultado_partido(fila):
    if fila['HT_SCORE'] > fila['AW_SCORE']:
        return 'Victoria local'
    elif fila['HT_SCORE'] < fila['AW_SCORE']:
        return 'Victoria visitante'
    else: 
        return 'Empate'
    
#Aplicar funcion para resultado de cada partido
df_todos['RESULT'] = df_todos.apply(resultado_partido, axis=1)

#Separar partidos por equipo
local = df_todos.groupby('HOME_TEAM').agg({'HT_SCORE': ['sum'], 'AW_SCORE': ['sum'], 'RESULT': ['count', lambda x: (x == 'Victoria local').sum(), lambda x: (x == 'Empate').sum(), lambda x: (x == 'Victoria Visitante').sum()]})
visitante = df_todos.groupby('AWAY_TEAM').agg({'AW_SCORE': ['sum'], 'HT_SCORE': ['sum'], 'RESULT': ['count', lambda x: (x == 'Victoria visitante').sum(), lambda x: (x == 'Empate').sum(), lambda x: (x == 'Victoria local').sum()]})

#Renombrar columnas
local.columns = ['Goles a favor', 'Goles en contra', 'Partidos jugados', 'Victorias', 'Empates', 'Derrotas']
visitante.columns = ['Goles a favor', 'Goles en contra', 'Partidos jugados', 'Victorias', 'Empates', 'Derrotas']

#Unificar estadisticas local y visitante
estadisticas = local.add(visitante, fill_value=0)

#Calular porcentajes de victoria, empate y derrota
estadisticas['Porcentaje Victoria'] = (estadisticas['Victorias'] / estadisticas['Partidos jugados']) * 100
estadisticas['Porcentaje Empate'] = (estadisticas['Empates'] / estadisticas['Partidos jugados']) * 100
estadisticas['Porcentaje Derrota'] = (estadisticas['Derrotas'] / estadisticas['Partidos jugados']) * 100

estadisticas.reset_index(inplace=True)
estadisticas.rename(columns={'index': 'Equipo'}, inplace=True)

#Ordenar por porcentaje de victoria
estadisticas.sort_values(by='Porcentaje Victoria', ascending=False, inplace=True)

estadisticas.head(20)



,HOME_TEAM,Goles a favor,Goles en contra,Partidos jugados,Victorias,Empates,Derrotas,Porcentaje Victoria,Porcentaje Empate,Porcentaje Derrota
13,Bayern München,192,65,71,52,9,6,73.239437,12.676056,8.450704
48,Manchester City,175,75,75,48,14,8,64.000000,18.666667,10.666667
62,Real Madrid,174,97,79,49,12,11,62.025316,15.189873,13.924051
44,Liverpool FC,144,72,65,40,9,11,61.538462,13.846154,16.923077
38,Juventus,102,65,63,37,9,9,58.730159,14.285714,14.285714
25,FC Barcelona,116,73,61,34,13,9,55.737705,21.311475,14.754098
56,Paris Saint-Germain,143,76,63,35,11,10,55.555556,17.460317,15.873016
19,Chelsea FC,85,51,49,27,10,7,55.102041,20.408163,14.285714
42,Leicester City,11,10,10,5,2,3,50.000000,20.000000,30.000000
4,AFC Ajax,74,50,38,19,7,3,50.000000,18.421053,7.894737


### Visualizaciones

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#Configuracion seaborn
sns.set(style='whitegrid')